<a href="https://colab.research.google.com/github/nimasha1228/Data_Ingestion/blob/main/Data_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the Dataset 

The dataset was taken from the G-Research Crypto Forecasting competition. The csv file is in google drive and could be downloaded by following code. 

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

FILE_PATH = "/content/Data/train.csv.zip"
# Link: https://drive.google.com/file/d/1-sZwK6sq5BgI99hH-WNgJpKz42cgEcme/view?usp=sharing

gdd.download_file_from_google_drive(file_id='1-sZwK6sq5BgI99hH-WNgJpKz42cgEcme',
                                    dest_path=FILE_PATH,
                                    unzip=True)

Unzipping...Done.


In [ ]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_dataset(filepath, method='pandas'):
    if method == 'pandas':
        ds = pd.read_csv(filepath)
        return ds


def read_config_file(filepath):

    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0


Writing testutility.py


**Read Data File Using Pandas, Dask, Modin, Ray**

**Pandas**

In [ ]:
import pandas as pd

In [ ]:
%%time
df = pd.read_csv('/content/Data/train.csv')
print(df.head())

    timestamp  Asset_ID  Count  ...       Volume          VWAP    Target
0  1514764860         2   40.0  ...    19.233005   2373.116392 -0.004218
1  1514764860         0    5.0  ...    78.380000      8.530000 -0.014399
2  1514764860         1  229.0  ...    31.550062  13827.062093 -0.014643
3  1514764860         5   32.0  ...  6626.713370      7.657713 -0.013922
4  1514764860         7    5.0  ...   121.087310     25.891363 -0.008264

[5 rows x 10 columns]
CPU times: user 29.5 s, sys: 4.46 s, total: 34 s
Wall time: 33.9 s


--------- End -----------

**Modin**

In [ ]:
!pip install "modin[ray]" # Install Modin dependencies and Ray to run on Ray

     |████████████████████████████████| 761 kB 5.0 MB/s 
     |████████████████████████████████| 133 kB 51.5 MB/s 
     |████████████████████████████████| 11.3 MB 49.4 MB/s 
     |████████████████████████████████| 57.6 MB 1.3 MB/s 
     |████████████████████████████████| 171 kB 55.7 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 128 kB 55.8 MB/s 
     |████████████████████████████████| 3.0 MB 56.3 MB/s 
     |████████████████████████████████| 192 kB 54.0 MB/s 
     |████████████████████████████████| 1.1 MB 51.5 MB/s 
     |████████████████████████████████| 201 kB 75.4 MB/s 
     |████████████████████████████████| 82 kB 208 kB/s 
     |████████████████████████████████| 160 kB 66.5 MB/s 
     |████████████████████████████████| 271 kB 75.7 MB/s 
     |████████████████████████████████| 85 kB 3.3 MB/s 
     |████████████████████████████████| 57 kB 4.5 MB/s 
  Created wheel for gpustat: filename=gpustat-1.0.0b1-py3-none-any.whl size=1597

In [ ]:
!pip install "modin[dask]" # Install Modin dependencies and Dask to run on Dask

     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 822 kB 49.0 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Attempting uninstall: dask
    Found existing installation: dask 2.12.0
    Uninstalling dask-2.12.0:
      Successfully uninstalled dask-2.12.0
  Attempting uninstall: distributed
    Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.

Restart Runtime and Run the code

In [ ]:
import modin.pandas as pd

In [ ]:
%%time
df = pd.read_csv('/content/Data/train.csv')
print(df.head()) 

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

(deploy_ray_func pid=1289) tcmalloc: large alloc 1409646592 bytes == 0x56130e660000 @  0x7f99c0ec31e7 0x56130ae4af98 0x56130ae15e27 0x56130aef79be 0x56130af2e802 0x56130ae1b7da 0x56130ae8e18e 0x56130ae879ee 0x56130ae1abda 0x56130ae8cd00 0x56130ae879ee 0x56130ad59e2b 0x56130ae89fe4 0x56130ae879ee 0x56130ad59e2b 0x56130ae89fe4 0x56130ad59d14 0x7f99bdcaebc2 0x7f99bdd5d6fc 0x7f99bdcb5696 0x7f99bde4b3bb 0x7f99bdda6c7f 0x7f99bde72533 0x7f99bde7322a 0x7f99bde7ea7e 0x7f99bde5c25b 0x7f99be056806 0x7f99be42bcfb 0x7f99be42cf51 0x7f99be42d3d0 0x7f99bde576d0
(deploy_ray_func pid=1288) tcmalloc: large alloc 1409646592 bytes == 0x557d4f694000 @  0x7f522c3061e7 0x557d4c317f98 0x557d4c2e2e27 0x557d4c3c49be 0x557d4c3fb802 0x557d4c2e87da 0x557d4c35b18e 0x557d4c3549ee 0x557d4c2e7bda 0x557d4c359d00 0x557d4c3549ee 0x557d4c226e2b 0x557d4c356fe4 0x557d4c3549ee 0x557d4c226e2b 0x557d4c356fe4 

    timestamp  Asset_ID  Count  ...       Volume          VWAP    Target
0  1514764860         2   40.0  ...    19.233005   2373.116392 -0.004218
1  1514764860         0    5.0  ...    78.380000      8.530000 -0.014399
2  1514764860         1  229.0  ...    31.550062  13827.062093 -0.014643
3  1514764860         5   32.0  ...  6626.713370      7.657713 -0.013922
4  1514764860         7    5.0  ...   121.087310     25.891363 -0.008264

[5 rows x 10 columns]
CPU times: user 4.65 s, sys: 4.2 s, total: 8.84 s
Wall time: 57.1 s


**Ray**

*Restart Runtime and Run the code*

In [ ]:
import os
os.environ["MODIN_ENGINE"] = "ray"

In [ ]:
import modin.pandas as pd

In [ ]:
%%time
df = pd.read_csv('/content/Data/train.csv')
print(df.head()) 

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

(deploy_ray_func pid=885) tcmalloc: large alloc 1409646592 bytes == 0x55685e31e000 @  0x7f6eb88df1e7 0x55685b99ef98 0x55685b969e27 0x55685ba4b9be 0x55685ba82802 0x55685b96f7da 0x55685b9e218e 0x55685b9db9ee 0x55685b96ebda 0x55685b9e0d00 0x55685b9db9ee 0x55685b8ade2b 0x55685b9ddfe4 0x55685b9db9ee 0x55685b8ade2b 0x55685b9ddfe4 0x55685b8add14 0x7f6eb56cabc2 0x7f6eb57796fc 0x7f6eb56d1696 0x7f6eb58673bb 0x7f6eb57c2c7f 0x7f6eb588e533 0x7f6eb588f22a 0x7f6eb589aa7e 0x7f6eb587825b 0x7f6eb5a72806 0x7f6eb5e47cfb 0x7f6eb5e48f51 0x7f6eb5e493d0 0x7f6eb58736d0
(deploy_ray_func pid=884) tcmalloc: large alloc 1409646592 bytes == 0x55a11e39c000 @  0x7f5826f1f1e7 0x55a11b4ddf98 0x55a11b4a8e27 0x55a11b58a9be 0x55a11b5c1802 0x55a11b4ae7da 0x55a11b52118e 0x55a11b51a9ee 0x55a11b4adbda 0x55a11b51fd00 0x55a11b51a9ee 0x55a11b3ece2b 0x55a11b51cfe4 0x55a11b51a9ee 0x55a11b3ece2b 0x55a11b51cfe4 0x

    timestamp  Asset_ID  Count  ...       Volume          VWAP    Target
0  1514764860         2   40.0  ...    19.233005   2373.116392 -0.004218
1  1514764860         0    5.0  ...    78.380000      8.530000 -0.014399
2  1514764860         1  229.0  ...    31.550062  13827.062093 -0.014643
3  1514764860         5   32.0  ...  6626.713370      7.657713 -0.013922
4  1514764860         7    5.0  ...   121.087310     25.891363 -0.008264

[5 rows x 10 columns]
CPU times: user 4.81 s, sys: 4.8 s, total: 9.61 s
Wall time: 1min


------------ End ------------

**Dask**

*Restart Run time and Run the code*

In [ ]:
import os
os.environ["MODIN_ENGINE"] = "dask"

In [ ]:
import modin.pandas as pd

In [ ]:
%%time
df = pd.read_csv('/content/Data/train.csv')
print(df.head())

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()



    timestamp  Asset_ID  Count  ...       Volume          VWAP    Target
0  1514764860         2   40.0  ...    19.233005   2373.116392 -0.004218
1  1514764860         0    5.0  ...    78.380000      8.530000 -0.014399
2  1514764860         1  229.0  ...    31.550062  13827.062093 -0.014643
3  1514764860         5   32.0  ...  6626.713370      7.657713 -0.013922
4  1514764860         7    5.0  ...   121.087310     25.891363 -0.008264

[5 rows x 10 columns]
CPU times: user 7.51 s, sys: 15.5 s, total: 23 s
Wall time: 1min 3s


In [ ]:
df.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


----------- End -------------

**yaml file**

In [ ]:
%%writefile file.yaml
file_type: csv
file_name: train
table_name: train
output_file_name: out
output_file_type: txt
inbound_delimiter: ","
outbound_delimiter: "|"
columns:
    - timestamp
    - Asset_ID
    - Count
    - Open
    - High
    - Low
    - Close
    - Volume
    - VWAP
    - Target

Overwriting file.yaml


In [ ]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [ ]:
config_data['inbound_delimiter']

','

In [ ]:
#inspecting data of config file
config_data

{'columns': ['timestamp',
  'Asset_ID',
  'Count',
  'Open',
  'High',
  'Low',
  'Close',
  'Volume',
  'VWAP',
  'Target'],
 'file_name': 'train',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'output_file_name': 'out',
 'output_file_type': 'txt',
 'table_name': 'train'}

In [ ]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("/content/Data/train.csv",delimiter=',')
df_sample.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [ ]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./Data/" + config_data['file_name'] + f'.{file_type}'
print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

 ./Data/train.csv


,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [ ]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [ ]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['timestamp', 'asset_id', 'count', 'open', 'high', 'low', 'close',
       'volume', 'vwap', 'target'],
      dtype='object')
columns of YAML are: ['timestamp', 'Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Target']


In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


In [ ]:
output_file_path = './' +config_data['output_file_name'] +'.'+config_data['output_file_type']+'.'+'gz'
output_file_path

'./out.txt.gz'

In [ ]:
df.to_csv(output_file_path, sep=config_data['outbound_delimiter'],compression='gzip')

**Summary of the file**

In [ ]:
num_cols = len(df.columns)
num_rows = df.shape[0]
print("Number of columns: ",num_cols)
print("Number of Rows: ",num_rows)
df.info(memory_usage=True)


Number of columns:  10
Number of Rows:  24236806
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24236806 entries, 0 to 24236805
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   asset_id   int64  
 2   count      float64
 3   open       float64
 4   high       float64
 5   low        float64
 6   close      float64
 7   volume     float64
 8   vwap       float64
 9   target     float64
dtypes: float64(8), int64(2)
memory usage: 1.8 GB
